In [1]:
from tensorflow import keras
import tensorflow as tf
print(tf.__version__)

2.9.2


In [2]:
from tensorflow.keras import layers

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

## Aquisição e visualização da base de dados

Este é problema de classificação que consiste em definir a qualidade de um carro com base nas características. 

Existem 4 classes (ruim, médio, bom e muito bom) e 6 atributos (se foi muito comprado, manutencao recente, número de portas, número de pessoas, tamanho do bagageiro e segurança)

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data

In [5]:
car = pd.read_csv('car.data', header=None)

In [6]:
car.columns = ['compras','manutencao','numero_de_portas','numero_de_pessoas','tamanho_bagageiro','seguranca','qualidade']

In [7]:
car.head()

,compras,manutencao,numero_de_portas,numero_de_pessoas,tamanho_bagageiro,seguranca,qualidade
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


## Pré-processamento da base de dados

In [8]:
car['qualidade'].replace({'unacc':0, 'acc':1, 'good':2, 'vgood':3}, inplace=True)

In [9]:
X = car.iloc[:,:-1]
Y = car.iloc[:,-1]

In [10]:
X = pd.get_dummies(X)

In [11]:
## Usando a técnica SMOTE para resolver o problema de dados desbalanceados
from imblearn.over_sampling import SMOTE

In [ ]:
strategy = {0:1250, 1:1250, 2:1250, 3:1250}
oversample = SMOTE(sampling_strategy=strategy)
X, Y = oversample.fit_resample(X, Y)

In [13]:
X[X > 1] = 1

## Separação da base em conjuntos de treino e teste 

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=93)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Definição dos dados marcados e métricas de performance

Vamos fazer o ajuste para One-Hot dos dados das classes. Temos 4 classes (0,1,2 e 3)

In [16]:
from tensorflow.keras.utils import to_categorical

In [17]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
print(y_train.shape)
print(y_test.shape)

(4000, 4)
(1000, 4)


A métrica de performance que será usada é a acurácia

## Escolha e construção da arquitetura profunda

Foi usada a arquitetura de redes MultiLayer Perceptron. 

Para o primeiro experimento, foi usada apenas 1 camada escondida com 13 neurônios. A função de ativação foi a função sigmóide.


Para o segundo experimento, foram usadas duas camadas escondidas (uma com 15 neurônios e outra com 9 neurônios). A função de ativação nas duas camadas foi a função relu. 

Para o terceiro experimento, foram usadas três camadas escondidas (com 16, 12 e 8 neurônios respectivamente). A função de ativação nas três camadas foi a função relu. 

O número de neurônios nas camadas de entradas dos três experimentos foi de 21, já que este é o número de atributos do problema. Já o número de neurônios nas camadas de saída foi de 4, já que este é o número de classes do problema.

## Execução dos experimentos

### Experimento 1

In [19]:
mod = keras.Sequential(name="MLP_1")
mod.add(keras.Input(shape=(21,))) 
mod.add(layers.Dense(13, activation="sigmoid"))
mod.add(layers.Dense(4, activation="softmax"))

In [20]:
mod.summary()

Model: "MLP_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                286       
                                                                 
 dense_1 (Dense)             (None, 4)                 56        
                                                                 
Total params: 342
Trainable params: 342
Non-trainable params: 0
_________________________________________________________________


In [21]:
mod.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [22]:
# Primeira execução
mod.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 1.4059 - accuracy: 0.3036 - val_loss: 1.3532 - val_accuracy: 0.3925
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 1.3268 - accuracy: 0.4825 - val_loss: 1.3138 - val_accuracy: 0.4725
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 1.2796 - accuracy: 0.5689 - val_loss: 1.2629 - val_accuracy: 0.5475
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 1.2239 - accuracy: 0.6147 - val_loss: 1.2061 - val_accuracy: 0.5825
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 1.1601 - accuracy: 0.6514 - val_loss: 1.1432 - val_accuracy: 0.6150
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 1.0888 - accuracy: 0.6800 - val_loss: 1.0765 - val_accuracy: 0.6375
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 1.0155 - accuracy: 0.7008 - val_loss: 1.0072 - val_accuracy: 0.6625
Epoch 

In [23]:
_, accuracy = mod.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.8047 - accuracy: 0.7670
Accuracy: 76.70


In [24]:
# Segunda execução
mod.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 2s 3ms/step - loss: 0.7653 - accuracy: 0.7906 - val_loss: 0.7787 - val_accuracy: 0.8075
Epoch 2/10
450/450 [==============================] - 1s 3ms/step - loss: 0.7173 - accuracy: 0.8197 - val_loss: 0.7344 - val_accuracy: 0.8125
Epoch 3/10
450/450 [==============================] - 2s 3ms/step - loss: 0.6742 - accuracy: 0.8333 - val_loss: 0.6943 - val_accuracy: 0.8300
Epoch 4/10
450/450 [==============================] - 2s 4ms/step - loss: 0.6356 - accuracy: 0.8492 - val_loss: 0.6565 - val_accuracy: 0.8450
Epoch 5/10
450/450 [==============================] - 1s 3ms/step - loss: 0.6008 - accuracy: 0.8647 - val_loss: 0.6234 - val_accuracy: 0.8550
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.5690 - accuracy: 0.8725 - val_loss: 0.5943 - val_accuracy: 0.8525
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.5410 - accuracy: 0.8831 - val_loss: 0.5644 - val_accuracy: 0.8725
Epoch 

In [25]:
_, accuracy = mod.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.4695 - accuracy: 0.8910
Accuracy: 89.10


In [26]:
# Terceira execução
mod.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 0.4512 - accuracy: 0.8967 - val_loss: 0.4721 - val_accuracy: 0.9025
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 0.4332 - accuracy: 0.8994 - val_loss: 0.4557 - val_accuracy: 0.8950
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.4172 - accuracy: 0.8981 - val_loss: 0.4392 - val_accuracy: 0.8975
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.4024 - accuracy: 0.9003 - val_loss: 0.4212 - val_accuracy: 0.9075
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.3890 - accuracy: 0.9061 - val_loss: 0.4085 - val_accuracy: 0.9075
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.3762 - accuracy: 0.9108 - val_loss: 0.3949 - val_accuracy: 0.9175
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.3649 - accuracy: 0.9150 - val_loss: 0.3842 - val_accuracy: 0.9075
Epoch 

In [27]:
_, accuracy = mod.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.9060
Accuracy: 90.60


Média da acurácia nas três execuções: 85,46%

Desvio padrão da acurácia nas três execuções: 7,63

### Experimento 2

In [28]:
mod2 = keras.Sequential(name="MLP_2")
mod2.add(keras.Input(shape=(21,))) 
mod2.add(layers.Dense(15, activation="relu"))
mod2.add(layers.Dense(9, activation="relu"))
mod2.add(layers.Dense(4, activation="softmax"))

In [29]:
mod2.summary()

Model: "MLP_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 15)                330       
                                                                 
 dense_3 (Dense)             (None, 9)                 144       
                                                                 
 dense_4 (Dense)             (None, 4)                 40        
                                                                 
Total params: 514
Trainable params: 514
Non-trainable params: 0
_________________________________________________________________


In [30]:
mod2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [31]:
# Primeira execução
mod2.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 1.3422 - accuracy: 0.3856 - val_loss: 1.2561 - val_accuracy: 0.5075
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 1.1400 - accuracy: 0.5600 - val_loss: 0.9850 - val_accuracy: 0.6600
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.7928 - accuracy: 0.7808 - val_loss: 0.6731 - val_accuracy: 0.8175
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.4991 - accuracy: 0.8772 - val_loss: 0.4618 - val_accuracy: 0.8750
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.3452 - accuracy: 0.9192 - val_loss: 0.3376 - val_accuracy: 0.9300
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.2574 - accuracy: 0.9397 - val_loss: 0.2588 - val_accuracy: 0.9325
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.2043 - accuracy: 0.9525 - val_loss: 0.2149 - val_accuracy: 0.9425
Epoch 

In [32]:
_, accuracy = mod2.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.1237 - accuracy: 0.9770
Accuracy: 97.70


In [33]:
# Segunda execução
mod2.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 0.1154 - accuracy: 0.9769 - val_loss: 0.1195 - val_accuracy: 0.9725
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 0.1057 - accuracy: 0.9794 - val_loss: 0.1105 - val_accuracy: 0.9750
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0975 - accuracy: 0.9808 - val_loss: 0.1019 - val_accuracy: 0.9775
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0905 - accuracy: 0.9828 - val_loss: 0.0928 - val_accuracy: 0.9725
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0848 - accuracy: 0.9808 - val_loss: 0.0887 - val_accuracy: 0.9775
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0795 - accuracy: 0.9833 - val_loss: 0.0829 - val_accuracy: 0.9775
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0747 - accuracy: 0.9844 - val_loss: 0.0765 - val_accuracy: 0.9800
Epoch 

In [34]:
_, accuracy = mod2.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.0608 - accuracy: 0.9850
Accuracy: 98.50


In [35]:
# Terceira execução
mod2.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0596 - accuracy: 0.9872 - val_loss: 0.0603 - val_accuracy: 0.9900
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0568 - accuracy: 0.9864 - val_loss: 0.0569 - val_accuracy: 0.9900
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0534 - accuracy: 0.9908 - val_loss: 0.0557 - val_accuracy: 0.9875
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0510 - accuracy: 0.9900 - val_loss: 0.0543 - val_accuracy: 0.9900
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0492 - accuracy: 0.9903 - val_loss: 0.0517 - val_accuracy: 0.9900
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0475 - accuracy: 0.9903 - val_loss: 0.0491 - val_accuracy: 0.9900
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0453 - accuracy: 0.9911 - val_loss: 0.0467 - val_accuracy: 0.9900
Epoch 

In [36]:
_, accuracy = mod2.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.0405 - accuracy: 0.9880
Accuracy: 98.80


Média da acurácia nas três execuções: 98,33%

Desvio padrão da acurácia nas três execuções: 0,57

### Experimento 3

In [37]:
mod3 = keras.Sequential(name="FirstDeepModel")
mod3.add(keras.Input(shape=(21,))) 
mod3.add(layers.Dense(16, activation="relu"))
mod3.add(layers.Dense(12, activation="relu"))
mod3.add(layers.Dense(8, activation="relu"))
mod3.add(layers.Dense(4, activation="softmax"))

In [38]:
mod3.summary()

Model: "FirstDeepModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                352       
                                                                 
 dense_6 (Dense)             (None, 12)                204       
                                                                 
 dense_7 (Dense)             (None, 8)                 104       
                                                                 
 dense_8 (Dense)             (None, 4)                 36        
                                                                 
Total params: 696
Trainable params: 696
Non-trainable params: 0
_________________________________________________________________


In [39]:
mod3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [40]:
# Primeira execução
mod3.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 1.3270 - accuracy: 0.3736 - val_loss: 1.2398 - val_accuracy: 0.4025
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 1.0213 - accuracy: 0.5556 - val_loss: 0.8580 - val_accuracy: 0.6475
Epoch 3/10
450/450 [==============================] - 2s 4ms/step - loss: 0.6426 - accuracy: 0.7661 - val_loss: 0.5116 - val_accuracy: 0.8425
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.3591 - accuracy: 0.8967 - val_loss: 0.3170 - val_accuracy: 0.8700
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.2231 - accuracy: 0.9308 - val_loss: 0.2050 - val_accuracy: 0.9500
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.1677 - accuracy: 0.9500 - val_loss: 0.1607 - val_accuracy: 0.9500
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.1387 - accuracy: 0.9589 - val_loss: 0.1335 - val_accuracy: 0.9500
Epoch 

In [41]:
_, accuracy = mod3.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.0899 - accuracy: 0.9750
Accuracy: 97.50


In [42]:
# Segunda execução
mod3.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0807 - accuracy: 0.9836 - val_loss: 0.0732 - val_accuracy: 0.9850
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0697 - accuracy: 0.9847 - val_loss: 0.0672 - val_accuracy: 0.9875
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0651 - accuracy: 0.9883 - val_loss: 0.0550 - val_accuracy: 0.9925
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0582 - accuracy: 0.9894 - val_loss: 0.0508 - val_accuracy: 0.9900
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0562 - accuracy: 0.9883 - val_loss: 0.0547 - val_accuracy: 0.9875
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0525 - accuracy: 0.9878 - val_loss: 0.0604 - val_accuracy: 0.9875
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0487 - accuracy: 0.9892 - val_loss: 0.0448 - val_accuracy: 0.9875
Epoch 

In [43]:
_, accuracy = mod3.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.0346 - accuracy: 0.9950
Accuracy: 99.50


In [44]:
# Terceira execução
mod3.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0386 - accuracy: 0.9928 - val_loss: 0.0319 - val_accuracy: 0.9925
Epoch 2/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0364 - accuracy: 0.9944 - val_loss: 0.0323 - val_accuracy: 0.9950
Epoch 3/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0342 - accuracy: 0.9939 - val_loss: 0.0316 - val_accuracy: 0.9900
Epoch 4/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0338 - accuracy: 0.9933 - val_loss: 0.0283 - val_accuracy: 0.9925
Epoch 5/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0334 - accuracy: 0.9933 - val_loss: 0.0271 - val_accuracy: 0.9950
Epoch 6/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0308 - accuracy: 0.9947 - val_loss: 0.0243 - val_accuracy: 0.9950
Epoch 7/10
450/450 [==============================] - 1s 2ms/step - loss: 0.0310 - accuracy: 0.9950 - val_loss: 0.0255 - val_accuracy: 0.9925
Epoch 

In [45]:
_, accuracy = mod3.evaluate(X_test, y_test, batch_size=8)
print('Accuracy: %.2f' % (accuracy*100))

125/125 [==============================] - 0s 1ms/step - loss: 0.0254 - accuracy: 0.9970
Accuracy: 99.70


Média da acurácia nas três execuções: 98,9%

Desvio padrão da acurácia nas três execuções: 1,21

## Avaliação dos resultados

Avaliando os resultados da acurácia nos três experimentos, pode-se perceber que o experimento que teve a média de acurácia mais alta foi o que teve a arquitetura de rede mais complexa com três camadas de neurônios, embora seja importante citar que a média de acurácia na predição para os três experimentos foi maior que 85%.

O tempo de treino e de predição não variou nos 3 experimentos, com um tempo de 1 segundo e 2 milissegundos por época em cada execução de treino e 1 milissegundo por cada execução na predição.

Quanto ao número de parâmetros treináveis, o primeiro experimento teve uma rede com 342 parâmetros treináveis (286 da camada escondida e 56 da camada final). O segundo experimento teve 514 parâmetros treináveis (330 da primeira camada escondida, 144 da segunda camada escondida e 40 da camada de saída). Por fim, o terceiro experimento teve uma rede com 696 parâmetros treináveis (352 na primeira camada escondida, 204 na segunda camada escondida, 104 na terceira camada escondida e 36 na camada de saída).